 # Desconsiderando os registros que possuam grande concordância ou pouquíssima concordância

# Seleção aleatória de conjuntos

In [5]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
import os, errno
import re
import numpy as np
import math
import pandas as pd
import sys
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# nao_passaram = 0
# iteracoes = 0

############################################################################################################################
#Definição de funções
############################################################################################################################

#Função para geração do conjunto de treinamento
def geraTrainSet(ct, dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
        
    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)     
    
#Função para geração do conjunto de teste
def geraTestSet(ct, dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    
    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)

#Função para geração do conjunto de treinamento e teste
#caso a seleção aleatória seja maior que o conjunto PC
def geraTrainSetVazio(dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
            
    with open(dir+file1+".csv", "w") as my_empty_csv:
      my_empty_csv.write('ERRO!')
    
def geraTestSetVazio(dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
            
    with open(dir+file1+".csv", "w") as my_empty_csv:
      my_empty_csv.write('ERRO!')        
    
    
############################

#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'med' #Estatística para ordenamento
#qtd_alg = 23 #Quantidade total de algoritmos (Modificado para ser atualizado em cada iteração de acordo com a quantidade máxima de algoritmos em cada experimento)
qtd_alg_nd = 2 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
# k = 3 #Tamanho da janela
# janelas = [2,3,4,5]
# janelas = [3] 
# janelas = [1, 0.01, 0.03, 0.05] # K = {1, 1%, 3%, 5%}
janelas = [0.01] #Quando houver a janela vencedora. Após testar as combinações acima
 #com todas as bases. Lembrando que só deve ser usada para as demais QP além e QP6
# orcamentos = [50, 100, 150, 200]
orcamentos = [150, 200]
tam_min_ct = 20
seed = 500
# nAlg = 23

etapa = '2 - AA[random]'

bases = ["cd"]
qps = ["qp4"] #Apenas qp4, na verdade

for base in bases:
    
    print("Base atual: {0}".format(base))
    
    for qp in qps:
        
        print("QP atual: {0}".format(qp))


        dirOrig = "../../csv/conjuntosDS/conjuntosDivergAA/"+base+"/"+qp+"/"
        dirEstat = "../../csv/estatisticas/"+base+"/"+qp+"/"
        estat = dirEstat+"estatisticaDS.csv"

        for orcamento in orcamentos:

            dirDest = "../../csv/conjuntosDS/treinoTeste/"+base+"/"+qp+"/"+"Random-"+str(orcamento)+"/"

            print("Orçamento atual: {0} ".format(orcamento))

            nao_passaram = 0
            iteracoes = 0

            estatisticas = pd.read_csv(estat, index_col=['algoritmosUtilizados', 'etapa', 'permutacao'], sep=';')

            ###### Criação do dataframe que armazenará as estatísticas dos conjuntos de treino e teste
            #   estat_conj = pd.DataFrame (index=['algUtl', 'permutacao'], columns=['algUtl','permutacao','tamConjTreino','prctgDup','prctgNaoDup','tamConjTeste'])
            estat_conj = pd.DataFrame (columns=['algUtl','permutacao','tamConjTreino','prctgDup','prctgNaoDup','tamConjTeste'])


            arquivos = [] #Adicionado depois

            for _, _, arquivo in os.walk(dirOrig):
                arquivos.extend(arquivo)   
                
            print(arquivos)

            for arq in arquivos:
                
                print(arq)

                if ('_NEW' in arq):# & (iteracoes <= 50):
#                 if ('diverg(10)1_NEW' in arq): # & (iteracoes <= 0): #Aqui, Diego!
#                 if (True): # & (iteracoes <= 0): #Aqui, Diego!
                    #         if ('diverg(10)18_NEW' in arq) & (iteracoes <= 0):
                    #         if ('diverg(10)18_NEW' in arq) | ('diverg(10)15_NEW' in arq) & (iteracoes <= 1):
                    print("##################################################################")
                    print("Analisando o arquivo: {0}".format(arq))
                    print("##################################################################")

                    iteracoes += 1

                    num = re.sub(r'diverg.*\)', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
                    num = num.replace('_NEW.csv','')

                    algUtl = re.sub(r'diverg.*\(', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
                    algUtl = re.sub(r'\).*', r'', algUtl) #Alterar para fazer a substituição de tudo em uma linha só
                    algUtl = int(algUtl)

                    permutacao = int(num)
                    print("permutacao: {0}".format(permutacao))

                    linhaAtual = estatisticas.xs((algUtl, '1 - acm diverg', permutacao))    

                    ###### Leitura do conjunto de pares conflitantes
                    pc = dirOrig+arq
                    
#                     print(pc)

                    pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes

                    cols = list(pc.columns.values)
                    cols.pop(cols.index('duplicata'))
                    pc = pc[['duplicata']+cols]

                    pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
                    pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 

                    duplicata = pc_aa.loc[:, 'duplicata' ]

                    #Adicionando a coluna de duplicatas a pc_vetores
                    pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False)

                    #             conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

#                     conj_treino = pc_vetores.sample(orcamento, random_state=500)
                    if(len(pc_vetores) == orcamento):
        
                        print("Ei seu bixim: {0}".format(len(pc_vetores)))
        
                        estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
                        'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                        'fp', 'tn', 'fn'] ] = (['ERRO-Insuf', 1, orcamento,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

                        algUtl = str(algUtl)

                        geraTrainSetVazio(dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')
                        geraTestSetVazio(dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')
                        estat_conj.loc[len(estat_conj.index)] = [algUtl, permutacao, 0, 0, 0, 0]

                        continue
                    
                    
                    #Apenas para criar o conjunto treino inicialmente
                    conj_treino = pc_vetores.sample(10, random_state=500)
                    orcamentoAux = orcamento
                    
                    try:
                        #O verdadeiro conj_treino deve ter o valor gerado aqui
                        conj_treino = pc_vetores.sample(orcamento, random_state=500)
                    except ValueError:
                        print("Quantidade de elementos insuficientes para realizar seleção aleatória!")
                        print("Tamanho de pc_vetores: {0} - Tamanho da amostra: {1}". format(len(pc_vetores), orcamentoAux))
                        
                        estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
                        'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                        'fp', 'tn', 'fn'] ] = (['ERRO-Insuf', 1, orcamentoAux,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

                        algUtl = str(algUtl)

                        geraTrainSetVazio(dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')
                        geraTestSetVazio(dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')
                        estat_conj.loc[len(estat_conj.index)] = [algUtl, permutacao, 0, 0, 0, 0]

                        continue

                    iteracao = 1
                    inspecoesManuais = orcamento

                    duplicatas = [i for i in conj_treino.duplicata if i == True]
                    duplicatas = duplicatas.count(True)
                    percentDup = float(duplicatas/float(len(conj_treino))*100)
                    percentDup = float("{0:.2f}".format(percentDup))
        #             print("{0}% de duplicatas.".format(str(percentage)[:5]))

                    nao_duplicatas = [i for i in conj_treino.duplicata if i == False]
                    nao_duplicatas = nao_duplicatas.count(False)
                    percentNaoDup = float(nao_duplicatas/float(len(conj_treino))*100)
                    percentNaoDup = float("{0:.2f}".format(percentNaoDup))

                    abordagem = 'DS'

                    da = linhaAtual['da'].item()
                    dm = duplicatas
                    ndm = nao_duplicatas

                    tp = float(linhaAtual['tp'].item() + dm)
                    fp = float(linhaAtual['fp'].item())
                    tn = float(linhaAtual['tn'].item())# + ndm) #Retirado
                    fn = float(linhaAtual['fn'].item() - dm) #Adicionado

                    precision = tp/(tp+fp)
                    recall = tp/(tp+fn)
                    fmeasure = 2*((precision*recall)/(precision+recall))

                        #Adicionando valor à última linha
                    estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
                        'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                        'fp', 'tn', 'fn'] ] = ([abordagem, iteracao, inspecoesManuais, precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])
                        #algUtl = str(algUtl).replace('.0','')
                    algUtl = str(algUtl)

                    geraTrainSet(conj_treino, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')

                    #Resta para compor o conjunto teste tudo aquilo que está em pc_vetores, mas não em conj_treino
                    conj_teste = pc_vetores.loc[pc_vetores.index.difference(conj_treino.index)]
                    geraTestSet(conj_teste, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')

                    estat_conj.loc[len(estat_conj.index)] = [algUtl, permutacao, len(conj_treino), percentDup, percentNaoDup, len(conj_teste)]


            ############################################################################################################################
            #Estatísticas
            ############################################################################################################################
            #Para voltar o dataframe ao normal
            estatisticas = estatisticas.reset_index(level=['algoritmosUtilizados', 'etapa', 'permutacao'])

            estatisticas = estatisticas[['abordagem', 'etapa', 'algoritmosUtilizados', 'permutacao', 'iteracao', 'inspecoesManuais', 'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']]

            estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']] = \
            estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']].astype(int)

            estatisticas.to_csv(dirEstat+'estatisticaDS2-Random'+str(orcamento)+'.csv', sep=';', index=False)   

            estat_conj.to_csv(dirEstat+'estatisticasConjuntos-Random'+str(orcamento)+'.csv', sep=';', index=False)   

Base atual: cd
QP atual: qp4
Orçamento atual: 50 
['diverg(5)93_NEW.csv']
diverg(5)93_NEW.csv
##################################################################
Analisando o arquivo: diverg(5)93_NEW.csv
##################################################################
permutacao: 93
Orçamento atual: 100 
['diverg(5)93_NEW.csv']
diverg(5)93_NEW.csv
##################################################################
Analisando o arquivo: diverg(5)93_NEW.csv
##################################################################
permutacao: 93
Ei seu bixim: 100
Orçamento atual: 150 
['diverg(5)93_NEW.csv']
diverg(5)93_NEW.csv
##################################################################
Analisando o arquivo: diverg(5)93_NEW.csv
##################################################################
permutacao: 93
Quantidade de elementos insuficientes para realizar seleção aleatória!
Tamanho de pc_vetores: 100 - Tamanho da amostra: 150
Orçamento atual: 200 
['diverg(5)93_NEW.csv']
diverg(5)93_N